In [2]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 159.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 184.1 MB/s eta 0:00:00


In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
from torch import nn
import torch

checkpoint = "lmsys/toxicchat-t5-large-v1.0"
device = "cuda"
seq2seq_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [2]:
class T5ForClassification(T5ForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels if hasattr(config, 'num_labels') else 2
        self.classification_head = nn.Linear(config.d_model, self.num_labels)
    
    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask, **kwargs)
        hidden_state = encoder_outputs.last_hidden_state  
        pooled_output = hidden_state[:, 0]  # Take [CLS] token
        logits = self.classification_head(pooled_output)
        loss = None

        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits.view(-1, self.num_labels), labels.view(-1))
        
        return {'logits': logits, 'loss': loss}

In [3]:
from transformers import AutoConfig
config = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).config
config.num_labels = 2

classification_model = T5ForClassification(config)
classification_model.load_state_dict(seq2seq_model.state_dict(), strict=False)

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from huggingface_hub import login

login()
repository_id = "inxoy/classification_t5_model"

classification_model.push_to_hub(repository_id)
tokenizer = AutoTokenizer.from_pretrained("t5-large")
tokenizer.push_to_hub(repository_id)

classification_model.save_pretrained("./classification_t5_model")
tokenizer = AutoTokenizer.from_pretrained("t5-large")  # Save tokenizer for consistency
tokenizer.save_pretrained("./classification_t5_model")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/inxoy/classification_t5_model/commit/b4339c75aa4f4cec1b923ff27db6c25968198dad', commit_message='Upload tokenizer', commit_description='', oid='b4339c75aa4f4cec1b923ff27db6c25968198dad', pr_url=None, repo_url=RepoUrl('https://huggingface.co/inxoy/classification_t5_model', endpoint='https://huggingface.co', repo_type='model', repo_id='inxoy/classification_t5_model'), pr_revision=None, pr_num=None)

In [ ]:
#  Reload as AutoModelForSequenceClassification
classification_model = AutoModelForSequenceClassification.from_pretrained("./classification_t5_model")
tokenizer = AutoTokenizer.from_pretrained("./classification_t5_model")